In [13]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import wordninja
from wordninja import LanguageModel
import re
import numpy as np
import gzip
import shutil

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

In [14]:
x = 'xxxapplexxx'
y = 'apple'
x = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
x

'xxx apple xxx'

# 資料讀取

In [15]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO']].dropna(axis=0)
df['string_X_train'] = df['string_X_train'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(lambda x:x.split('\n'))
for idx in tqdm(df.index):
    for p in df.loc[idx,'Y_label']:
        if p in df.loc[idx,'string_X_train']:
            df.loc[idx,'p'] = p
            continue
        else:
            df.loc[idx,'p'] = 'not find'
print(df[df['p']!='not find'].shape)
df = df[df['p']!='not find']
df['Y_label'] = df['p']
df = df.drop(['p'],axis=1)

for i in tqdm(df.index):
    x,y = df.loc[i,'string_X_train'],df.loc[i,'Y_label']
    df.loc[i,'string_X_train'] = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
print(df.shape)
df.head(10)

<ipython-input-15-a4745f83696b>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(df.index):



(1970, 4)


<ipython-input-15-a4745f83696b>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):



(1970, 3)


,string_X_train,Y_label,EXPNO
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18
7,COMMODITY ...,ORIENTED YARN,25
8,COMMODITY ...,ORIENTED YARN,25
9,COMMODITY ...,POLYESTER FILAMENT YARN,25
10,COMMODITY ...,ORIENTED YARN,25
11,COMMODITY ...,ORIENTED YARN,25
31,PRODUCT TETRAHYDROFURAN 998 PCT MINQUANTIT...,TETRAHYDROFURAN,2P
44,COMMODITY LLDPE TAISOX 3470 QUANTITY 11200 MT...,TAISOX 3470,18
62,HDPE TAISOX 9020 1000 MTUNIT PRICE USD106000M...,TAISOX 9020,18


In [16]:
# 起始結束位置標註
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys),ye_lst.append(ye)
    
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(1,df.shape)

df = df[df['Y_label']!=''] # 去掉空值
print(2,df.shape)

df = df.loc[df['string_Y_1']!=-1,:] # 去掉找不到答案的
print(3,df.shape)

df = df.dropna(subset=['string_X_train','Y_label'],axis=0) # 去掉NAN
print(4,df.shape)

df.head(10)

1 (1970, 5)
2 (1970, 5)
3 (1970, 5)
4 (1970, 5)


,string_X_train,Y_label,EXPNO,string_Y_1,string_Y_2
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,17,28
7,COMMODITY ...,ORIENTED YARN,25,121,134
8,COMMODITY ...,ORIENTED YARN,25,121,134
9,COMMODITY ...,POLYESTER FILAMENT YARN,25,147,170
10,COMMODITY ...,ORIENTED YARN,25,121,134
11,COMMODITY ...,ORIENTED YARN,25,121,134
31,PRODUCT TETRAHYDROFURAN 998 PCT MINQUANTIT...,TETRAHYDROFURAN,2P,11,26
44,COMMODITY LLDPE TAISOX 3470 QUANTITY 11200 MT...,TAISOX 3470,18,17,28
62,HDPE TAISOX 9020 1000 MTUNIT PRICE USD106000M...,TAISOX 9020,18,6,17


In [17]:
# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a==b

<ipython-input-17-4ec65d08a61a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


In [18]:
from termcolor import colored

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
1 COMMODITY EVA TAISOX 7350M QUANTITY 50 MT  UNIT PRICE USD 2555 MT SUBTOTAL PRICES USD 127750 PACKING STANDARD EXPORT PACKING ORIGIN TAIWAN PRICE TERM CIF CHINA MAIN PORT CONTRACT NO H0DV232 COMMODITY HDPE   TAISOX 9001   QUANTITY 100 MT  UNIT PRICE USD 1145 MT SUBTOTAL PRICES USD 114500 PACKING STANDARD EXPORT PACKING ORIGIN TAIWAN PRICE TERM CIF CHINA MAIN PORT CONTRACT NO H0DV353 COMMODITY HDPE TAISOX 8010 QUANTITY 600 MT  UNIT PRICE USD 1145 MT SUBTOTAL PRICES USD 687000 PACKING STANDARD EXPORT PACKING ORIGIN TAIWAN PRICE TERM CIF CHINA MAIN PORT CONTRACT NO H0DV364 COMMODITY HDPE TAISOX 9001 QUANTITY 700 MT  UNIT PRICE USD 1145 MT SUBTOTAL PRICES USD 801500 PACKING STANDARD EXPORT PACKING ORIGIN TAIWAN PRICE TERM CIF CHINA MAIN PORT CONTRACT NO H0DV37
TAISOX 9001
1
1800 MT   PHTHALIC ANHYDRIDE   PA AT THE RATE OF USD 102000 PER MT AND OTHER DETAILS AS PER BENEFICIARYS PROFORMA INVOICE  NO NP1448 DATED 06042021 CFR KARACHI PORT PAKISTAN INCOTERMS 2020
PHTHALIC ANHYDRIDE
2
ITEM 1

In [19]:
# 保存
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')

(1970, 5)
